In [1]:
import pandas as pd
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
# from sklearn.preprocessing
import keras
import os
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense,Conv2D,MaxPooling2D,Dropout,Flatten

EPOCHS=20
NUM_CLASSES=43
CURR_PATH=os.getcwd()

In [2]:
data=[]
labels=[]
fileData=pd.read_csv(CURR_PATH+"/Dataset/Train.csv")
labelDict=pd.read_csv(CURR_PATH+"/Dataset/Meta.csv")
labelDict=labelDict.drop("ShapeId",axis=1).drop("ColorId",axis=1).drop("SignId",axis=1)
labelDict={i:val for i,val in zip(labelDict["ClassId"],labelDict["Path"])}
# fileData.shape #(39209,8)
for i in range(fileData.shape[0]):
    img=Image.open(CURR_PATH+"/Dataset/"+fileData["Path"][i])
    img=img.resize((30,30))
    img=np.array(img,dtype=np.float32)/255.0
    data.append(img)
    labels.append(fileData["ClassId"][i])
# plt.imshow(data[7888])
# plt.imshow(Image.open(CURR_PATH+"/Dataset/"+labelDict[labels[7888]]))

In [3]:
data=np.array(data)
labels=np.array(labels)
# data.shape,labels.shape #((39209, 30, 30, 3), (39209,))

In [4]:
#output toh NUM_CLASSES honge na labels ka shape toh abhi
#   (39209,) hai. isko OHE karke (39209,NUM_CLASSES)
from keras.utils import to_categorical
labels=to_categorical(labels,NUM_CLASSES)
# print(labels.shape) #(39209, 43)

In [6]:
model=Sequential()
model.add(keras.Input(data.shape[1:]))
model.add(Conv2D(32,(5,5),activation="relu"))
model.add(Conv2D(32,(5,5),activation="relu"))
model.add(MaxPooling2D((2,2)))
model.add(Dropout(rate=0.2))
model.add(Conv2D(64,(3,3),activation="relu"))
model.add(Conv2D(64,(3,3),activation="relu"))
model.add(MaxPooling2D((2,2)))
model.add(Dropout(rate=0.2))

In [7]:
model.add(Flatten())#None,3,3,64->None,3*3*64
model.add(Dense(576,activation="relu"))
model.add(Dropout(rate=0.4))
model.add(Dense(NUM_CLASSES,activation="softmax"))


In [ ]:
# model.compile(
#     loss="categorical_crossentropy",
#     optimizer="adam",
#     metrics=["accuracy"]
# )
# from keras.preprocessing.image import ImageDataGenerator
# datagen = ImageDataGenerator(
#     rotation_range=10,
#     width_shift_range=0.1,
#     height_shift_range=0.1,
#     shear_range=0.1,
#     zoom_range=0.1,
#     brightness_range=[0.8, 1.2],
#     horizontal_flip=False,  # Don't flip traffic signs horizontally
#     fill_mode='nearest'
# )

# trainHistory=model.fit(
#     datagen.flow(train_images, train_images_labels, batch_size=32),
#     epochs=EPOCHS,
#     steps_per_epoch=len(train_images) // 32,
#     validation_data=(test_images, test_images_labels)
# )
# model.save(f"./MODELS/RNN_Traffic_{model_choice}4.keras")

In [8]:
model.compile(
    loss="categorical_crossentropy",
    optimizer="adam",
    metrics=["accuracy"]
)
trainHistory=model.fit(
    data,
    labels,
    batch_size=32,
    epochs=20
)
# print(labels.shape)

Epoch 1/20
1226/1226 ━━━━━━━━━━━━━━━━━━━━ 36s 28ms/step - accuracy: 0.5666 - loss: 1.5953
Epoch 2/20
1226/1226 ━━━━━━━━━━━━━━━━━━━━ 34s 28ms/step - accuracy: 0.9622 - loss: 0.1270
Epoch 3/20
 217/1226 ━━━━━━━━━━━━━━━━━━━━ 27s 27ms/step - accuracy: 0.9808 - loss: 0.0607

KeyboardInterrupt: 

In [ ]:
model.save(CURR_PATH+"/model.keras")

In [ ]:
from keras.models import load_model
model=load_model("./model.keras")

In [ ]:
#prediction
import pandas as pd
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
test_imgPaths=pd.read_csv("./Dataset/Test.csv")["Path"][:333]
test_labels=pd.read_csv("./Dataset/Test.csv")["ClassId"][:333]
def test(test_imgPaths):
    accuracyList=[]
    predList=[]
    for i in range(len(test_imgPaths)):
        img=Image.open("./Dataset/"+test_imgPaths[i])
        img=img.resize((30,30))
        img=np.array(img,dtype=np.float32)/255.0
        img=np.expand_dims(img,axis=0)
        prediction_ohe=model.predict(img)[0]
        pred=np.argmax(prediction_ohe)
        predList.append(pred)
        accuracyList.append(int(pred==test_labels[i]))
    return accuracyList, predList

accuracy_list,pred=test(test_imgPaths)
print("accuracy: ",sum(accuracy_list)/len(test_labels)*100,"%")
print(accuracy_list)
print("l",test_labels,"l")
print("b ",pred)
# print()

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━